In [ ]:
from sklearn.impute import KNNImputer


In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler,QuantileTransformer,KBinsDiscretizer
import glob
import imblearn
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

In [ ]:
knn =  KNNImputer(n_neighbors=2)


In [ ]:
train=  pd.read_csv("/home/kruzr/personal_git/data/hackeearth/Dataset/train.csv")
test =  pd.read_csv("/home/kruzr/personal_git/data/hackeearth/Dataset/test.csv")

In [5]:
train["condition"] = knn.fit_transform(train[["condition"]])

In [6]:
test["condition"] = knn.transform(test[["condition"]])

In [7]:
train_x_cols = ["pet_id","issue_date","listing_date","condition","color_type","length(m)","height(cm)","X1","X2"]
train_y_cols = ["breed_category","pet_category"]

In [8]:
train_x =  train[train.columns.difference(train_y_cols)]
train_y =  train[train_y_cols]

In [9]:
train_y["breed_category"] = train_y["breed_category"].astype(int)

/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
# train_mms_len =  StandardScaler()
# train_mms_he  =  StandardScaler()

train_mms_len =  KBinsDiscretizer(n_bins=7,encode="ordinal")


In [12]:
def feature_engine(df,test=False):
    df['issue_year'] = pd.DatetimeIndex(df['issue_date']).year
    df['listing_year'] = pd.DatetimeIndex(df['listing_date']).year
    df['issue_month'] = pd.DatetimeIndex(df['issue_date']).month
    df['listing_month'] = pd.DatetimeIndex(df['listing_date']).month
    
    df["age"] = df["listing_year"]-df["issue_year"]
    df["x1x2"] = df["X1"]*df["X2"]
    df["x1px2"] = df["X1"]+df["X2"]
    df["lh"] = df["length(m)"]*100*df["height(cm)"]
    
    df = pd.get_dummies(df, columns=['color_type'])
    df = pd.get_dummies(df, columns=['X1'])
    df = pd.get_dummies(df, columns=['X2'])
    df = pd.get_dummies(df, columns=['issue_year'])
    df = pd.get_dummies(df, columns=['listing_year'])
    df = pd.get_dummies(df, columns=['issue_month'])
    df = pd.get_dummies(df, columns=['listing_month'])
    
    if( not test):
        df[["length(m)","height(cm)","x1x2","lh","x1px2","age"]] = train_mms_len.fit_transform(df[["length(m)","height(cm)","x1x2","lh","x1px2","age"]])
        df["condition"] = knn.fit_transform(df[["condition"]])
    else:
        df[["length(m)","height(cm)","x1x2","lh","x1px2","age"]] = train_mms_len.transform(df[["length(m)","height(cm)","x1x2","lh","x1px2","age"]])
        df["condition"] = knn.transform(df[["condition"]])
        
    df = pd.get_dummies(df, columns=['length(m)'])
    df = pd.get_dummies(df, columns=["height(cm)"])
    df = pd.get_dummies(df, columns=["x1x2"])
    df = pd.get_dummies(df, columns=["lh"])
    df = pd.get_dummies(df, columns=["x1px2"])
    df = pd.get_dummies(df, columns=['age'])
        
    return df

In [13]:
fe_train_x = feature_engine(train_x,test=False)

/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/kruzr/work/virtual_envs/notebookenv/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/kruzr/work/virtual_envs/notebookenv/lib/pyt

In [14]:
fe_train_x.columns

Index(['condition', 'issue_date', 'listing_date', 'pet_id',
       'color_type_Agouti', 'color_type_Apricot', 'color_type_Black',
       'color_type_Black Brindle', 'color_type_Black Smoke',
       'color_type_Black Tabby',
       ...
       'lh_5.0', 'lh_6.0', 'x1px2_0.0', 'x1px2_1.0', 'x1px2_2.0', 'x1px2_3.0',
       'age_0.0', 'age_1.0', 'age_2.0', 'age_3.0'],
      dtype='object', length=177)

In [15]:
fe_test_x  = feature_engine(test,test = True)

In [16]:
fe_test_x.columns

Index(['pet_id', 'issue_date', 'listing_date', 'condition',
       'color_type_Agouti', 'color_type_Apricot', 'color_type_Black',
       'color_type_Black Brindle', 'color_type_Black Smoke',
       'color_type_Black Tabby',
       ...
       'lh_5.0', 'lh_6.0', 'x1px2_0.0', 'x1px2_1.0', 'x1px2_2.0', 'x1px2_3.0',
       'age_0.0', 'age_1.0', 'age_2.0', 'age_3.0'],
      dtype='object', length=172)

In [17]:
missing_list = list(set(fe_train_x.columns)-set(fe_test_x.columns))

In [96]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import ExtraTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [18]:
model_breed    = XGBClassifier(n_estimators=1500,n_jobs=-1,learning_rate=.05,verbosity=1)
model_pet_cat  = XGBClassifier(n_estimators=1500,n_jobs=-1,learning_rate=.05,verbosity=1)
# from sklearn.svm import SVC
# model_breed    = SVC()
# model_pet_cat  = SVC()
# model_breed    = CatBoostClassifier(iterations=2000,learning_rate=.01)
# model_pet_cat  = CatBoostClassifier(iterations=2000,learning_rate=.01)
# model_breed    = XGBClassifier(n_estimators=1500,n_jobs=-1,class_weight="balanced_subsample")
# model_pet_cat  = XGBClassifier(n_estimators=1500,n_jobs=-1,class_weight="balanced_subsample")
# model_breed  = ExtraTreeClassifier()
# model_pet_cat =  ExtraTreeClassifier()
# model_breed    = LogisticRegression()
# model_pet_cat  = LogisticRegression()

In [19]:
use_cols = fe_train_x.columns.difference(["issue_date","listing_date","pet_id"])
for col in missing_list:
    fe_test_x[col] = 0
model_breed.fit(fe_train_x[use_cols].values,train_y["breed_category"].values)
model_pet_cat.fit(fe_train_x[use_cols].values,train_y["pet_category"].values)

XGBClassifier(learning_rate=0.05, n_estimators=1500, n_jobs=-1,
              objective='multi:softprob')

In [ ]:
submission_df = pd.DataFrame()
submission_df["pet_id"] =  fe_test_x["pet_id"]
submission_df["breed_category"] = model_breed.predict(fe_test_x[use_cols].values)
submission_df["pet_category"]   = model_pet_cat.predict(fe_test_x[use_cols].values)

In [ ]:
submission_df.to_csv("submission_xgb50.csv",index=False)
#35 best

# Inference

In [ ]:
pd.crosstab(submission_df["pet_category"],submission_df["breed_category"])

In [98]:
def ensemble_submissions():
    sub_list = glob.glob("*.csv")
    print(sub_list)
    list_df = []
    for sub in sub_list:
        df =  pd.read_csv(sub)
        list_df.append(df)
    final_df =  pd.DataFrame()
    final_df["pet_id"] = list_df[0]["pet_id"]
    ensemble_submission = list_df[0]
    for sub in list_df[1:]:
        ensemble_submission =  pd.merge(ensemble_submission,sub,on="pet_id")
    column_list         = ensemble_submission.columns.to_list()
    pet_catergory_list  = [x for x in column_list if((x.split("_")[0]=="pet")&(x.split("_")[-1]!="id"))]
    breed_category_list  = [x for x in column_list if((x.split("_")[0]=="breed"))]
    final_df["breed_category"] = ensemble_submission[breed_category_list].mode(axis=1)[0].values
    final_df["pet_category"]   = ensemble_submission[pet_catergory_list].mode(axis=1)[0].values
    return final_df


In [99]:
fdf = ensemble_submissions()

['submission_1.csv', 'submission_35.csv', 'submission_gnb47.csv', 'submission_xgb50.csv', 'Submission_fold_4.csv', 'ensemble_sub_5_sample.csv', 'submission_cat41.csv', 'ensemble_ensemble_sample_fin1.csv', 'submission_14.csv', 'ensemble_ensemble_sample.csv', 'submission_28.csv', 'submission_37.csv', 'submission_38.csv', 'Submission_fold_breed_2.csv', 'submission_33.csv', 'submission_2.csv', 'submission_25.csv', 'ensemble_latest2_sample.csv', 'submission_cat42.csv', 'Submission_fold_1.csv', 'ensemble_sub3.csv', 'submission_13.csv', 'submission_36.csv', 'submission_11.csv', 'Submission_fold_0.csv', 'Submission_fold_pet_4.csv', 'Submission_fold_pet_1.csv', 'Submission_fold_breed_3.csv', 'oversample_pet.csv', 'submission_4.csv', 'submission_31.csv', 'submission_34.csv', 'submission_15.csv', 'submission_40.csv', 'ensemble_ensemble_fold.csv', 'ensemble_ensemble_sample_fin3.csv', 'ensemble_ensemble_underoversample.csv', 'submission_39.csv', 'submission_30.csv', 'Submission_fold_breed_1.csv', '

In [100]:
fdf.to_csv("ensemble_all.csv",index=False)
#best ensemble_ensemble_fold_breed

In [20]:
imp_df = pd.DataFrame({"columns":fe_train_x.columns.difference(["pet_id","issue_date","listing_date"]),"impor_br":model_breed.feature_importances_,"impor_pc":model_pet_cat.feature_importances_})

In [21]:
pcr = imp_df.sort_values("impor_pc",ascending=False)

In [95]:
pcr.impor_br.describe()

count    174.000000
mean       0.005747
std        0.038186
min        0.000000
25%        0.000384
50%        0.001293
75%        0.002440
max        0.414624
Name: impor_br, dtype: float64

In [72]:
feature_pc = pcr["columns"].to_list()[:75]

In [ ]:
feature_br

In [45]:
br = imp_df.sort_values("impor_br",ascending=False)

In [70]:
len(feature_pc)

100

In [56]:
feature_br = br["columns"].to_list()[:100]

In [ ]:
feature_br

In [ ]:
imp_df

In [ ]:
#convert continous features to bins
#try catboost
#try kfold CV and train multiple models
#try oversampled model small class pc
#prepare documentation

In [ ]:
train.pet_category.value_counts()

In [77]:
from imblearn.over_sampling import SMOTE

In [78]:
sm = SMOTE()

In [ ]:
print(train_x_cols )

In [ ]:
train_x,train_y = sm.fit_resample(fe_train_x[use_cols].values,train["pet_category"].values)

In [ ]:
list(train_y).count(0),list(train_y).count(4),list(train_y).count(2),list(train_y).count(1)

In [ ]:
xgb_oversample =  XGBClassifier(n_estimators=300,n_jobs=-1,learning_rate=.05,verbosity=1)

In [ ]:
xgb_oversample.fit(train_x,train_y)

In [80]:
sub =  pd.read_csv("ensemble_ensemble_fold_breed.csv")

In [ ]:
sub["pet_category"] = xgb_oversample.predict(fe_test_x[use_cols].values)

In [ ]:
sub.to_csv("undersample_pet.csv",index=False)

In [27]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [28]:
skf =  StratifiedKFold(n_splits=5)

In [30]:
from sklearn.metrics import classification_report

In [74]:
X = fe_train_x[feature_pc].values

In [75]:
y = train["pet_category"].values

In [97]:
model_list  = []
i= 1
for train_ix, test_ix in skf.split(X, y):
    print("Running Process for Fold {}".format(i))
    i+=1
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    model_1 = CatBoostClassifier(iterations=2000,learning_rate=.01)

    train_X,train_y = sm.fit_resample(train_X,train_y)
    model_1.fit(train_X,train_y)
    model_list.append(model_1)    
    print(classification_report(test_y,model_1.predict(test_X)))

Running Process for Fold 1
0:	learn: 1.3737831	total: 74.1ms	remaining: 2m 28s
1:	learn: 1.3606437	total: 95.4ms	remaining: 1m 35s
2:	learn: 1.3489670	total: 116ms	remaining: 1m 16s
3:	learn: 1.3369181	total: 136ms	remaining: 1m 7s
4:	learn: 1.3258179	total: 156ms	remaining: 1m 2s
5:	learn: 1.3141970	total: 177ms	remaining: 58.7s
6:	learn: 1.3030996	total: 201ms	remaining: 57.3s
7:	learn: 1.2921637	total: 222ms	remaining: 55.3s
8:	learn: 1.2813878	total: 243ms	remaining: 53.7s
9:	learn: 1.2712136	total: 263ms	remaining: 52.2s
10:	learn: 1.2620033	total: 284ms	remaining: 51.4s
11:	learn: 1.2519490	total: 310ms	remaining: 51.4s
12:	learn: 1.2420502	total: 334ms	remaining: 51.1s
13:	learn: 1.2333628	total: 358ms	remaining: 50.8s
14:	learn: 1.2239574	total: 383ms	remaining: 50.7s
15:	learn: 1.2157657	total: 404ms	remaining: 50s
16:	learn: 1.2067884	total: 424ms	remaining: 49.5s
17:	learn: 1.1978934	total: 444ms	remaining: 48.9s
18:	learn: 1.1900895	total: 472ms	remaining: 49.2s
19:	learn: 

166:	learn: 0.6801216	total: 3.97s	remaining: 43.6s
167:	learn: 0.6788905	total: 3.99s	remaining: 43.5s
168:	learn: 0.6774593	total: 4.01s	remaining: 43.5s
169:	learn: 0.6758326	total: 4.03s	remaining: 43.4s
170:	learn: 0.6742589	total: 4.05s	remaining: 43.3s
171:	learn: 0.6730816	total: 4.07s	remaining: 43.2s
172:	learn: 0.6716588	total: 4.09s	remaining: 43.2s
173:	learn: 0.6703102	total: 4.11s	remaining: 43.1s
174:	learn: 0.6691938	total: 4.13s	remaining: 43s
175:	learn: 0.6680630	total: 4.14s	remaining: 43s
176:	learn: 0.6669785	total: 4.16s	remaining: 42.9s
177:	learn: 0.6657722	total: 4.18s	remaining: 42.8s
178:	learn: 0.6646701	total: 4.21s	remaining: 42.8s
179:	learn: 0.6630189	total: 4.23s	remaining: 42.8s
180:	learn: 0.6615160	total: 4.25s	remaining: 42.7s
181:	learn: 0.6603450	total: 4.27s	remaining: 42.6s
182:	learn: 0.6591890	total: 4.29s	remaining: 42.6s
183:	learn: 0.6581426	total: 4.3s	remaining: 42.5s
184:	learn: 0.6568708	total: 4.32s	remaining: 42.4s
185:	learn: 0.655

326:	learn: 0.5224327	total: 7.36s	remaining: 37.7s
327:	learn: 0.5215007	total: 7.38s	remaining: 37.6s
328:	learn: 0.5209760	total: 7.4s	remaining: 37.6s
329:	learn: 0.5202274	total: 7.42s	remaining: 37.6s
330:	learn: 0.5196771	total: 7.44s	remaining: 37.5s
331:	learn: 0.5191529	total: 7.46s	remaining: 37.5s
332:	learn: 0.5186768	total: 7.48s	remaining: 37.4s
333:	learn: 0.5179191	total: 7.5s	remaining: 37.4s
334:	learn: 0.5174754	total: 7.51s	remaining: 37.3s
335:	learn: 0.5164454	total: 7.54s	remaining: 37.3s
336:	learn: 0.5157969	total: 7.56s	remaining: 37.3s
337:	learn: 0.5153839	total: 7.59s	remaining: 37.3s
338:	learn: 0.5148521	total: 7.62s	remaining: 37.3s
339:	learn: 0.5143779	total: 7.63s	remaining: 37.3s
340:	learn: 0.5138722	total: 7.65s	remaining: 37.2s
341:	learn: 0.5133489	total: 7.67s	remaining: 37.2s
342:	learn: 0.5127821	total: 7.69s	remaining: 37.2s
343:	learn: 0.5115970	total: 7.72s	remaining: 37.1s
344:	learn: 0.5111186	total: 7.74s	remaining: 37.1s
345:	learn: 0.

490:	learn: 0.4415998	total: 10.7s	remaining: 33s
491:	learn: 0.4410806	total: 10.8s	remaining: 33s
492:	learn: 0.4406385	total: 10.8s	remaining: 33s
493:	learn: 0.4401500	total: 10.8s	remaining: 33s
494:	learn: 0.4397529	total: 10.8s	remaining: 33s
495:	learn: 0.4394476	total: 10.9s	remaining: 32.9s
496:	learn: 0.4391716	total: 10.9s	remaining: 32.9s
497:	learn: 0.4388435	total: 10.9s	remaining: 32.9s
498:	learn: 0.4384970	total: 10.9s	remaining: 32.8s
499:	learn: 0.4382866	total: 10.9s	remaining: 32.8s
500:	learn: 0.4377291	total: 11s	remaining: 32.8s
501:	learn: 0.4372697	total: 11s	remaining: 32.8s
502:	learn: 0.4369853	total: 11s	remaining: 32.7s
503:	learn: 0.4366617	total: 11s	remaining: 32.7s
504:	learn: 0.4362986	total: 11s	remaining: 32.7s
505:	learn: 0.4359823	total: 11.1s	remaining: 32.6s
506:	learn: 0.4354128	total: 11.1s	remaining: 32.6s
507:	learn: 0.4350320	total: 11.1s	remaining: 32.6s
508:	learn: 0.4346208	total: 11.1s	remaining: 32.6s
509:	learn: 0.4343462	total: 11.

650:	learn: 0.3913845	total: 14.1s	remaining: 29.2s
651:	learn: 0.3912434	total: 14.1s	remaining: 29.2s
652:	learn: 0.3910702	total: 14.2s	remaining: 29.2s
653:	learn: 0.3909335	total: 14.2s	remaining: 29.2s
654:	learn: 0.3907526	total: 14.2s	remaining: 29.2s
655:	learn: 0.3906001	total: 14.2s	remaining: 29.1s
656:	learn: 0.3903891	total: 14.2s	remaining: 29.1s
657:	learn: 0.3902157	total: 14.3s	remaining: 29.1s
658:	learn: 0.3900155	total: 14.3s	remaining: 29.1s
659:	learn: 0.3897051	total: 14.3s	remaining: 29s
660:	learn: 0.3894080	total: 14.3s	remaining: 29s
661:	learn: 0.3891567	total: 14.3s	remaining: 29s
662:	learn: 0.3890130	total: 14.4s	remaining: 29s
663:	learn: 0.3886836	total: 14.4s	remaining: 28.9s
664:	learn: 0.3885078	total: 14.4s	remaining: 28.9s
665:	learn: 0.3883048	total: 14.4s	remaining: 28.9s
666:	learn: 0.3880261	total: 14.4s	remaining: 28.9s
667:	learn: 0.3876955	total: 14.5s	remaining: 28.8s
668:	learn: 0.3874806	total: 14.5s	remaining: 28.8s
669:	learn: 0.387273

...

1339:	learn: 0.2907912	total: 32.5s	remaining: 16s
1340:	learn: 0.2906304	total: 32.5s	remaining: 16s
1341:	learn: 0.2905778	total: 32.5s	remaining: 15.9s
1342:	learn: 0.2904692	total: 32.5s	remaining: 15.9s
1343:	learn: 0.2903813	total: 32.6s	remaining: 15.9s
1344:	learn: 0.2901433	total: 32.6s	remaining: 15.9s
1345:	learn: 0.2900933	total: 32.6s	remaining: 15.8s
1346:	learn: 0.2899323	total: 32.6s	remaining: 15.8s
1347:	learn: 0.2897026	total: 32.6s	remaining: 15.8s
1348:	learn: 0.2896011	total: 32.7s	remaining: 15.8s
1349:	learn: 0.2895474	total: 32.7s	remaining: 15.8s
1350:	learn: 0.2895073	total: 32.7s	remaining: 15.7s
1351:	learn: 0.2894290	total: 32.8s	remaining: 15.7s
1352:	learn: 0.2893442	total: 32.8s	remaining: 15.7s
1353:	learn: 0.2892390	total: 32.8s	remaining: 15.7s
1354:	learn: 0.2891704	total: 32.8s	remaining: 15.6s
1355:	learn: 0.2890150	total: 32.8s	remaining: 15.6s
1356:	learn: 0.2889226	total: 32.9s	remaining: 15.6s
1357:	learn: 0.2888117	total: 32.9s	remaining: 15.

1501:	learn: 0.2746893	total: 36.6s	remaining: 12.1s
1502:	learn: 0.2745004	total: 36.6s	remaining: 12.1s
1503:	learn: 0.2742775	total: 36.6s	remaining: 12.1s
1504:	learn: 0.2741347	total: 36.7s	remaining: 12.1s
1505:	learn: 0.2740317	total: 36.7s	remaining: 12s
1506:	learn: 0.2739037	total: 36.7s	remaining: 12s
1507:	learn: 0.2737814	total: 36.7s	remaining: 12s
1508:	learn: 0.2737454	total: 36.8s	remaining: 12s
1509:	learn: 0.2736254	total: 36.8s	remaining: 11.9s
1510:	learn: 0.2735494	total: 36.8s	remaining: 11.9s
1511:	learn: 0.2735095	total: 36.8s	remaining: 11.9s
1512:	learn: 0.2734811	total: 36.9s	remaining: 11.9s
1513:	learn: 0.2733608	total: 36.9s	remaining: 11.8s
1514:	learn: 0.2732835	total: 36.9s	remaining: 11.8s
1515:	learn: 0.2731534	total: 36.9s	remaining: 11.8s
1516:	learn: 0.2730585	total: 36.9s	remaining: 11.8s
1517:	learn: 0.2729416	total: 37s	remaining: 11.7s
1518:	learn: 0.2728732	total: 37s	remaining: 11.7s
1519:	learn: 0.2728185	total: 37s	remaining: 11.7s
1520:	l

1662:	learn: 0.2600461	total: 40.6s	remaining: 8.24s
1663:	learn: 0.2599415	total: 40.7s	remaining: 8.21s
1664:	learn: 0.2598794	total: 40.7s	remaining: 8.19s
1665:	learn: 0.2598361	total: 40.7s	remaining: 8.17s
1666:	learn: 0.2596772	total: 40.8s	remaining: 8.14s
1667:	learn: 0.2595788	total: 40.8s	remaining: 8.12s
1668:	learn: 0.2594673	total: 40.8s	remaining: 8.1s
1669:	learn: 0.2594257	total: 40.8s	remaining: 8.07s
1670:	learn: 0.2593534	total: 40.9s	remaining: 8.05s
1671:	learn: 0.2592945	total: 40.9s	remaining: 8.02s
1672:	learn: 0.2592101	total: 40.9s	remaining: 8s
1673:	learn: 0.2591029	total: 40.9s	remaining: 7.97s
1674:	learn: 0.2589671	total: 41s	remaining: 7.95s
1675:	learn: 0.2588609	total: 41s	remaining: 7.92s
1676:	learn: 0.2586977	total: 41s	remaining: 7.9s
1677:	learn: 0.2585760	total: 41s	remaining: 7.87s
1678:	learn: 0.2584756	total: 41.1s	remaining: 7.85s
1679:	learn: 0.2583377	total: 41.1s	remaining: 7.83s
1680:	learn: 0.2582646	total: 41.1s	remaining: 7.8s
1681:	l

1824:	learn: 0.2477829	total: 44.7s	remaining: 4.28s
1825:	learn: 0.2476301	total: 44.7s	remaining: 4.26s
1826:	learn: 0.2475475	total: 44.7s	remaining: 4.24s
1827:	learn: 0.2474850	total: 44.8s	remaining: 4.21s
1828:	learn: 0.2474065	total: 44.8s	remaining: 4.19s
1829:	learn: 0.2472563	total: 44.8s	remaining: 4.16s
1830:	learn: 0.2472249	total: 44.8s	remaining: 4.14s
1831:	learn: 0.2471287	total: 44.8s	remaining: 4.11s
1832:	learn: 0.2470111	total: 44.9s	remaining: 4.09s
1833:	learn: 0.2469662	total: 44.9s	remaining: 4.06s
1834:	learn: 0.2468514	total: 44.9s	remaining: 4.04s
1835:	learn: 0.2467896	total: 44.9s	remaining: 4.01s
1836:	learn: 0.2467569	total: 45s	remaining: 3.99s
1837:	learn: 0.2467275	total: 45s	remaining: 3.96s
1838:	learn: 0.2466343	total: 45s	remaining: 3.94s
1839:	learn: 0.2465880	total: 45s	remaining: 3.92s
1840:	learn: 0.2465484	total: 45s	remaining: 3.89s
1841:	learn: 0.2463487	total: 45.1s	remaining: 3.87s
1842:	learn: 0.2462397	total: 45.1s	remaining: 3.84s
184

1987:	learn: 0.2364795	total: 48.5s	remaining: 293ms
1988:	learn: 0.2364366	total: 48.6s	remaining: 269ms
1989:	learn: 0.2364091	total: 48.6s	remaining: 244ms
1990:	learn: 0.2363703	total: 48.6s	remaining: 220ms
1991:	learn: 0.2362946	total: 48.6s	remaining: 195ms
1992:	learn: 0.2362489	total: 48.6s	remaining: 171ms
1993:	learn: 0.2362368	total: 48.7s	remaining: 146ms
1994:	learn: 0.2362158	total: 48.7s	remaining: 122ms
1995:	learn: 0.2361728	total: 48.7s	remaining: 97.6ms
1996:	learn: 0.2360952	total: 48.7s	remaining: 73.2ms
1997:	learn: 0.2360326	total: 48.8s	remaining: 48.8ms
1998:	learn: 0.2359691	total: 48.8s	remaining: 24.4ms
1999:	learn: 0.2359283	total: 48.8s	remaining: 0us
              precision    recall  f1-score   support

           0       0.38      0.71      0.49        17
           1       0.89      0.84      0.86      1436
           2       0.89      0.92      0.91      2125
           4       0.81      0.80      0.80       188

    accuracy                         

In [86]:
 for idx,model in enumerate(model_list):
        sub["pet_category"] =  model.predict(fe_test_x[feature_pc].values)
        sub.to_csv("Submission_fold_pet_{}.csv".format(idx),index=False)

In [ ]:
model_pet_cat.feature_importances_